## Using Web3 to interact with our smart contract

In [1]:
# Import libraries we will be using
import os
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account

from pathlib import Path
from getpass import getpass
import pandas as pd

In [2]:
#from etherscan import Etherscan
import json

In [3]:
#def admin():
load_dotenv()
#### Infura API suite provides instant access over HTTPS and WebSockets to the Ethereum network.
WEB3_INFURA_API_KEY =  os.getenv("WEB3_INFURA_API_KEY")
WEB3_INFURA_PROJECT_ID = os.getenv("WEB3_INFURA_PROJECT_ID")
# read contract for NFT musuem
nft_museum_address = os.getenv("nft_museum_address")
museum_ticket_address = os.getenv("museumEntry_contract")
### Contract NFT Art Musuem owner private key
museum_private_key = os.getenv("PRIVATE_KEY1")
ticket_private_key = os.getenv("PRIV_KEY")
#### project is deployed on Kovan Ethereum testnet
#ETHERSCAN_API = os.getenv("ETHERSCAN_API")
#eth = Etherscan(ETHERSCAN_API, net= "KOVAN")
#### Connect to infura kovan
https_str = f'https://kovan.infura.io/v3/{WEB3_INFURA_PROJECT_ID}'
w1 = Web3(Web3.HTTPProvider(https_str))
w1.middleware_onion.inject(geth_poa_middleware, layer=0)
account_contract_owner = Account.from_key(museum_private_key)
museum_contract_owner = Account.from_key(ticket_private_key)
#return WEB3_INFURA_API_KEY, WEB3_INFURA_PROJECT_ID, museum_private_key, eth, w1, account_contract_owner

### Open Contract ABI

In [4]:
with open("contract_abi.txt") as f:
    MuseumContract_json = json.load(f)

## Create an Instance with Museum Contract ABI

In [5]:
museum_contract_instance = w1.eth.contract(address=museum_ticket_address, abi=MuseumContract_json)

In [6]:
all_functions = museum_contract_instance.all_functions()

In [7]:
token_name = museum_contract_instance.functions.name().call()
token_symbol = museum_contract_instance.functions.symbol().call()
token_total_supply = museum_contract_instance.functions.totalSupply().call()

In [8]:
museum_contract_instance.functions.transfer

<Function transfer>

In [9]:
buyer_private_key = "3dbb034cc8d1faf58287aeceee6dee04aa47976666153e9e87cb89479ccb977e"

In [10]:
account_two = Account.from_key(buyer_private_key)
account_one= Account.from_key(ticket_private_key)

def create_raw_tx(account, recipient, amount):
    gasEstimate = w1.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w1.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w1.eth.getTransactionCount(account.address),
    }


def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w1.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()


print(account_one.address)
print(account_two.address)

0xE72b2C4391a8D1Bd1e6A98D96978bfaA20d02B62
0x5CA0fcF3c882a3b3fbFA93Db33C718A65bC55C41


In [13]:
nonce = w1.eth.get_transaction_count(account_one.address)
chain_id = w1.eth.chain_id
# For Gas get latest block for better estimation
w1.eth.getBlock("latest")
dict1 = w1.eth.getBlock("latest")
gasEstimate = dict1['gasUsed']
# we sometimes see that the Latest block could return 0 too
if gasEstimate < 600000:
    gasEstimate = 600000
gas_price = w1.eth.gasPrice

## Buy Ticket function, Transfer Buyers ETHER to Museum, Transfer Museum Token to Buyer.

In [14]:
def buyTicket(private_key):
    #define how much 1 ticket cost
    ticket_price = 1
    # check if seller has tikcets left
    tickets_remaining = museum_contract_instance.functions.balanceOf(account_one.address).call()
    if tickets_remaining > 0:
        
        # transfer ETHER from buyers private key to sellers private key
        buyer_account = Account.from_key(private_key)
        # the transaction might not go through if it doesnt dont send ticket
        tx_send = send_tx(buyer_account, account_one.address, ticket_price)
        nonce = w1.eth.get_transaction_count(account_one.address)
        chain_id = w1.eth.chain_id
        # For Gas get latest block for better estimation
        w1.eth.getBlock("latest")
        dict1 = w1.eth.getBlock("latest")
        gasEstimate = dict1['gasUsed']
        # we sometimes see that the Latest block could return 0 too
        if gasEstimate < 600000:
            gasEstimate = 600000
        gas_price = w1.eth.gasPrice
        print('nonce=', nonce )
        print('gasEstimate=', gasEstimate)
        print('gas_price=', gas_price)
        # transfer 1 MTKN to buyer
        tx_hash = museum_contract_instance.functions.transfer(buyer_account.address, 1).buildTransaction({'chainId':chain_id, 'gas': gasEstimate, 'gasPrice': gas_price, 'nonce': nonce})
        register_signed_txn = w1.eth.account.sign_transaction(tx_hash, private_key=ticket_private_key)
        register_sent_txn = w1.eth.send_raw_transaction(register_signed_txn.rawTransaction)
        return tx_send, register_sent_txn
        # museum_contract_instance.functions.balanceOf(buyer_account).call()
        # museum_contract_instance.functions.balanceOf(account_one).call()  

## Tesing the BuyTicket function 

In [16]:
third_buyer= "eb573233eed2422f33b9c1928f614a8d2a75d189fca2bd373469457eb205a241"

In [22]:
buyTicket(third_buyer)

0x6b85e225beae3735d8c2468c980771bfa34adc5b05d8d30101efb65e28f6d546
nonce= 40
gasEstimate= 1897446
gas_price= 5000000000


('0x6b85e225beae3735d8c2468c980771bfa34adc5b05d8d30101efb65e28f6d546',
 HexBytes('0x1485a02464bff15c8737de7b9931a68df110bcefec19b40b0b15fb4811b436e8'))

In [23]:
buyTicket("011167339039d7537ffbec9b628f73de70c0f443a95030d347aceeea6520a2e1")

0x4deb64b1ac147e8624c5e21b96390bc4b85fdbf87bdfedab55924b8b760cbaa4
nonce= 41
gasEstimate= 600000
gas_price= 5000000000


('0x4deb64b1ac147e8624c5e21b96390bc4b85fdbf87bdfedab55924b8b760cbaa4',
 HexBytes('0x4411529fa8efb58b9488a128d2af6ad8252f80f89d112ddf1fa7cee5109d3761'))

In [ ]:
buyTicket("")